In [58]:
import pandas as pd
import talib
import numpy as np
from math import e
from datetime import datetime, date
from pathlib import Path
import math
import os
from os import listdir
from os.path import isfile, join

In [59]:
def calc_new_col(row):
    if   (row['MA20'] ==True) and (row['RSI']==True) and (row['MacdVsSigLine']==True):# and (row['RS>RSma']== True) :
        return 'Buy'
    elif (row['MA20'] ==False) and (row['RSI']==False) and (row['MacdVsSigLine']==False):# and row['MacdVsSigLine']==False) :
        return 'Sell'
    else:
        return 'Neutral'


In [60]:
#function which calculates values of different technical  indicator . 
def ta_indicators(df):
#     df= df[['Date','Close']]
    df = df.copy()
    df['MA20V'] = talib.SMA(df['Close'],timeperiod=20)
    df['RSIV']= talib.RSI(df['Close'])  
    df['MACD'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
    df['MACDsig'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)[1]
    df.set_index('Date', inplace = True)
    return df

In [61]:
def parameters_result(df):
    df['MA20']= df['Close'] >  df['MA20V']
    df['RSI']=  df['RSIV']>50
    df['MacdVsSigLine']= talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]\
           > talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)[1]
#     df['+DI>-DI'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod = 14)\
#                     > talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod = 14)
    df["final"] = df.apply(calc_new_col, axis=1) #Getting final signal 
    return df

In [62]:
def scrip_backtest(simulated_df, simulations_tracker): 
    stock_wise_stat = []
    df_company = simulated_df
    historical_signal =  ['dummy', 'dummy']
    valid_signal = []

    ta_indicator_calculations = ta_indicators(df_company) #ta_indicators function which calculates TA indicator values in  a dataframe
    df = parameters_result(ta_indicator_calculations) # Returns true false based on parameters of TA in a dataframe
    
    df['P_change'] = df['Close'].pct_change()
    df['log'] = np.log(1+df['P_change'])
    annualized_BnH_stdev = df['log'].std() * math.sqrt(229)
    
    for_strategy_stdev = df.copy()
    for_strategy_stdev['final'] = for_strategy_stdev['final'].shift(1)
    for_strategy_stdev = for_strategy_stdev[for_strategy_stdev['final'].isin(['Buy'])]
    annualized_strategy_stdev = for_strategy_stdev['log'].std() * math.sqrt(229)

    for index_number, date in enumerate(list(df.index)):
        if index_number == 0:
            current_day_signal = 'Neutral'
            previous_day_signal = 'Neutral'
        else:
            previous_day_signal = df.loc[list(df.index)[index_number-1]]['final']
            current_day_signal = df.loc[list(df.index)[index_number]]['final']

        if current_day_signal == 'Buy' or current_day_signal == 'Sell':
            if current_day_signal == previous_day_signal:
                pass
            else:
                historical_signal.append(current_day_signal)
                if current_day_signal == historical_signal[-2]:#Buy signal can appear again after neutral signal so this codes removes that 
                    pass
                else:
                    signal_info = {'Date' : date,
                                   'Close': df.loc[list(df.index)[index_number]]['Close'],
                                   'Signal' : current_day_signal               
                                    }
                    valid_signal.append(signal_info)
    # ======================================================================================================================
    buy_sell_report = pd.DataFrame(valid_signal)

    if list(buy_sell_report['Signal'])[0] =="Sell":
        buy_sell_report = buy_sell_report[1:]
    elif list(buy_sell_report['Signal'])[-1]  == 'Buy':
            buy_sell_report = buy_sell_report[:-1]
    else:
        pass

    final_df = buy_sell_report
    final_df['Comission_adj_price'] = np.where(final_df['Signal'] == 'Buy',
                                           final_df['Close'] *  1.004,
                                           final_df['Close']*0.996 )
    final_df['CGT']  = np.where((final_df['Comission_adj_price'].diff()>0) & (final_df['Signal'] == 'Sell'),final_df['Comission_adj_price'].diff()*0.05, 0)
    final_df['CGT_adjustedPrice'] = final_df['Comission_adj_price'] - final_df['CGT']
    final_df['Strategy_return'] = final_df['CGT_adjustedPrice'].pct_change()
    final_df['Log_return'] = np.log(1+final_df['Strategy_return'])
    final_df.to_csv(f'D:/Thesis/Bullish phase backtesting results scripwise/{simulations_tracker}.csv')
    final_df = final_df[final_df['Signal'] == 'Sell']
    total_trades = final_df[final_df['Signal']=="Sell"].count()
    winning_trades = final_df[final_df['Strategy_return']>0].count()
    winning_rate = winning_trades/total_trades
    start_date = df.index[0]
    end_date = df.index[-1]
    years = (end_date - start_date).days/365
    buy_and_hold_cagr =  df['log'].sum()/years
    average_log_return = final_df['Log_return'].sum()/years
    cagr = average_log_return
#     buy_and_hold =  (list(df.Close)[-1]/list(df.Close)[0])**(1/years)-1
    
    performance_stats = {'Ticker' : f'{simulations_tracker}' ,
                         'Buy and Hold CAGR' : buy_and_hold_cagr,
                         'Strategy CAGR'  : cagr,
                         'Total Trades' : total_trades[0],
                         'Win rate' : round(winning_rate[0],2),
                        'Buy&H return annual std' : annualized_BnH_stdev,
                         'Strategy return annual std' : annualized_strategy_stdev,
                         'Period' : f'{round(years,2)} years'
                            }
    stock_wise_stat.append(performance_stats)
    return performance_stats

### Import Data

In [63]:
samples = pd.read_csv('D:/Thesis/Sample stocks.csv')

In [ ]:
all_scrip_stats = []
for index, ticker in enumerate(samples.Ticker):#in enumerate(samples.Ticker):
    print(ticker)
    price_history= pd.read_csv(f'D:/Thesis/adjusted prices/{ticker}.csv')
    price_history['Date'] = pd.to_datetime(price_history['Date'])
    stats = scrip_backtest(price_history,ticker )
    all_scrip_stats.append(stats)

In [69]:
x = pd.DataFrame.from_dict(all_scrip_stats)
x.to_csv('D:/Thesis/Bullish phase backtesting results scripwise/AllScripBacktest.csv')
print('Completed!')

Completed!
